In [1]:
import datasets
from datasets import load_dataset
import numpy as np
import pandas as pd
import os

C:\Users\DMIS\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset Load

In [ ]:
load_dataset("iwslt2017", 'iwslt2017-de-en', cache_dir='./data/')

### Extract text from XML Data

In [2]:
import xml.etree.ElementTree as ET


def make_dataset(file_path, docs) :
    f = open(file_path, "w", encoding='utf-8')
    for doc in docs :
        segs = doc.findall('seg')
        for seg in segs :
            text = seg.text.lower()
            try :
                f.write(text)
                f.write('\n')
            except : 
                print(text)
    f.close()

train_folder_path = "./data/de-en/training_and_development/"
test_folder_path = "./data/de-en/test/"

## train english
tree_en = ET.parse(train_folder_path + 'IWSLT17.TED.dev2010.de-en.en.xml')
root_en = tree_en.getroot()
refset = root_en.find('refset')
docs = refset.findall('doc')
make_dataset("./data/de-en/train.en", docs)

## train deutsch
tree_de = ET.parse(train_folder_path + "IWSLT17.TED.dev2010.de-en.de.xml")
root_de = tree_de.getroot()
srcset = root_de.find('srcset')
docs = srcset.findall('doc')
make_dataset("./data/de-en/train.de", docs)

## test en
tree_en_test = ET.parse(test_folder_path + 'IWSLT17.TED.tst2017.mltlng.en-de.en.xml')
root_en_test = tree_en_test.getroot()
srcset = root_en_test.find('srcset')
docs = srcset.findall('doc')
make_dataset("./data/de-en/test.en", docs)

## test deutsch
tree_de_test = ET.parse(test_folder_path + 'IWSLT17.TED.tst2017.mltlng.de-en.de.xml')
root_de_test = tree_de_test.getroot()
srcset = root_de_test.find('srcset')
docs = srcset.findall('doc')
make_dataset("./data/de-en/test.de", docs)



### Split Data Set

In [5]:
import xml.etree.ElementTree as ET
train_folder_path = "./data/de-en/training_and_development/"
test_folder_path = "./data/de-en/test/"

## train english
tree_en = ET.parse(train_folder_path + 'IWSLT17.TED.dev2010.de-en.en.xml')
root_en = tree_en.getroot()
refset = root_en.find('refset')
docs = refset.findall('doc')

len(docs)

8

### Make Vocabulary : BPE Encoding

In [7]:
import re

file_path = 'C://Users/DMIS/project/transformer/data/de-en/train.en'

## return (maximum counted pair, count)
## ex - (h st, 10)
def search_max_pair(dict) :
    pair_count = {}
    
    ## pair count
    for word in dict:
        unit_list = word.split(' ')
        count_of_word = dict[word] 
        for i in range(len(unit_list) - 1):
            new_word = unit_list[i] + ' ' + unit_list[i+1] ## insert space for distinguish frontword and backword
            pair_count[new_word] = pair_count[new_word] + count_of_word if (new_word in pair_count) else count_of_word
    ## search maximum pair
    max_count = 0
    max_pair = ''
    for pair in pair_count:
        if pair_count[pair] > max_count :
            max_count = pair_count[pair]
            max_pair = pair
    return (max_pair, max_count)

def merge_word_dict(dict, frontword, backword) :
    new_word_dict = {}
    for word in dict :
        unit_list = word.split(' ')
        new_word = ''
        for i in range(len(unit_list) - 1):
            if unit_list[i] != frontword or unit_list[i+1] != backword : 
                new_word = new_word + unit_list[i] + ' '
                continue
            new_word = new_word + unit_list[i] + unit_list[i+1] + ' '
            i = i + 1 ## skip next unit
        new_word_dict[new_word] = dict[word] ## change word with new key
    return new_word_dict
                
                
        

def byte_pair_encoding(file_path, count=10) : 
    f = open(file_path, "r", encoding='utf-8')
    stop_word_set = set([',', ' ', '-', '', '\n', '.'])
    word_dict = {} ##  key-bpe encoded words / value-count : word set for calculate maximun counted pair
    vocabulary = {} ## key-vacab word / value-count : vocabulary for encoding, 
    while True :
        line = f.readline()
        if not line :
            break
        
        words = re.split(' |, |. |! ', line)
        for word in words :
            if word not in stop_word_set :
                splited_word = ''
                for character in [*word]:
                    splited_word = splited_word + character + ' '
                    vocabulary[character] = vocabulary[character] + 1 if (character in vocabulary) else 1
                splited_word = splited_word + '</w>'
                word_dict[splited_word] = word_dict[splited_word] + 1 if (splited_word in word_dict) else 1
        #break
            
    for i in range(count) :
        max_pair, max_count = search_max_pair(word_dict)
        subwords = max_pair.split(' ')
        frontword = subwords[0]
        backword = subwords[1]
        
        word_dict = merge_word_dict(word_dict, frontword=frontword, backword=backword)
        vocabulary[frontword + backword] = max_count
        vocabulary[frontword] = vocabulary[frontword] - max_count
        vocabulary[backword] = vocabulary[backword] - max_count
        
byte_pair_encoding(file_path)
        

RuntimeError: dictionary keys changed during iteration